In [1]:
import pandas as pd
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime

In [5]:
# 채무자키 20409775 2301 > 2031년으로 수정해야함 안그럼 에러 아래 날짜변경에서 에러남##############
# 엑셀을 pickle파일로
wd = r'D:\3.자산\전산 dataset\230213'
for f in os.listdir(wd) :
    if os.path.isfile(join(wd,f)) :
        if re.search(".xlsx", f) :
            df = pd.read_excel(join(wd, f)).fillna("")
            df.to_pickle(join(wd, os.path.splitext(f)[0]+".pkl"))

In [6]:
# 파일 명칭 자동 저장
account_f, debt_f, grt_f, law_f, collection_f = "", "", "", "", ""
for root, dirs, files in os.walk(wd) :
    for f in files :
        if re.search(".pkl", f) :
            if re.search("계좌",f) : account_f = f
            elif re.search("채무자",f) : debt_f = f
            elif re.search("보증인",f) : grt_f = f
            elif re.search("법조치",f) : law_f = f
            elif re.search("입금",f) : collection_f = f

In [7]:
account = pd.read_pickle(join(wd, account_f))
debt = pd.read_pickle(join(wd, debt_f))
#collection = pd.read_pickle(join(wd,collection_f))
############################
# 매각사 = "전북은행"
# account = account[account.매각사.str.contains(매각사)]
# debt = debt[debt.매각사.str.contains(매각사)]
############################
print(len(account), len(debt))

78157 70795


In [8]:
# 채무자키 20409775 2301 > 2031년으로 수정해야함 안그럼 에러
account.시효완성일 = pd.to_datetime(account.시효완성일)
#collection.입금일 = pd.to_datetime(collection.입금일)

In [9]:
raw_data = pd.DataFrame(None)
raw_data["계좌키"] = account.계좌키.apply(str)
raw_data["채무자키"] = account.채무자키.apply(str)
raw_data["타채무자키"] = account.타채무자키.apply(str)

raw_data["채권구분"] = account.채권구분
#raw_data["매각사"] = account.매각사
raw_data["채무상태"] = account.채무상태
raw_data["담당자"] = account.담당자
raw_data["성명"] = account.채무자명

raw_data["주민등록번호"] = account.채무자주민번호.apply(lambda x : x[:8])
raw_data["계좌번호"] = account.계좌번호
raw_data["원채권사"] = account.원채권사
raw_data["매각사구분"] = account.매각사구분
raw_data["계정과목"] = account.계정과목
raw_data["최초원금"] = account.최초원금
raw_data["최초법비용"] = account.최초법비용
raw_data["최초미수이자"] = account.최초미수이자
raw_data["최초연체이자율"] = account.최초연체이자율
raw_data["현재원금"] = account.현재원금
raw_data["현재법비용"] = account.현재법비용
raw_data["OPB"] = account[["현재원금","현재법비용"]].sum(axis=1)
raw_data["현재미수이자"] = account.현재미수이자
raw_data["현재연체이자"] = account.현재연체이자
raw_data["현재합계"] = account.현재합계
raw_data["회수합계"] = account.회수합계
raw_data["최초대출일"] = account.최초대출일
raw_data["최초연체일"] = account.최초연체일 # 없는 경우도 있음
raw_data["매입일"] = account.매입일
raw_data["시효완성일"] = account.시효완성일

In [10]:
debt.채무자키 = debt.채무자키.apply(str)
debt = debt[["채무자키", "개인법인", "연령", "자택우편번호", "자택주소","보증인성명"]]
raw_data = pd.merge(raw_data, debt, on="채무자키", how="inner")
raw_data.개인법인.isnull().value_counts()

False    78157
Name: 개인법인, dtype: int64

In [11]:
# 전체 저장하기
raw_data.to_excel(join(wd, "2.솔림헬프_전체_원데이터_"+datetime.today().strftime("%Y%m%d")+".xlsx"), index=False)

In [15]:
# 특정 매각사만 저장하기
cond = "스마트저축-01" ###################################
raw_data[raw_data.매각사구분==cond].to_excel(join(wd, cond+"_"+datetime.today().strftime("%Y%m%d")+".xlsx"), index=False)

### 차입자산 POOL별 중복확인

In [82]:
dfdf = pd.read_excel(join(r"D:\3.자산\POOL별 관리자산\무차입자산","1.차입자산 리스트.xlsx"))

In [88]:
중복 = dfdf[dfdf.duplicated(['계좌키'],keep=False)].sort_values(by='계좌키')
#.to_excel(join(r"D:\3.자산\POOL별 관리자산\무차입자산","중복체크.xlsx"), index=False)

In [89]:
중복["계좌키"] = 중복["계좌키"].astype(int).astype(str)
중복["채무자키"] = 중복["채무자키"].astype(int).astype(str)

In [90]:
중복

,계좌키,채무자키,POOL_
22224,200919485,20407790,POOL_05
22223,200919485,20407790,POOL_05
26521,200919497,20407882,POOL_08
21292,200919497,20407882,POOL_05
21293,200919997,20407883,POOL_05
...,...,...,...
12525,201046260,20515407,POOL_13
12545,201046280,20515427,POOL_13
12546,201046280,20515427,POOL_13
12602,201046336,20515483,POOL_13


In [91]:
중복 = 중복.reset_index(drop=True, inplace=False)
중복

,계좌키,채무자키,POOL_
0,200919485,20407790,POOL_05
1,200919485,20407790,POOL_05
2,200919497,20407882,POOL_08
3,200919497,20407882,POOL_05
4,200919997,20407883,POOL_05
...,...,...,...
1725,201046260,20515407,POOL_13
1726,201046280,20515427,POOL_13
1727,201046280,20515427,POOL_13
1728,201046336,20515483,POOL_13


In [92]:
중복.to_excel(join(r"D:\3.자산\POOL별 관리자산\무차입자산","중복체크.xlsx"), index=False)

In [2]:
중복 = pd.read_excel(r'D:\3.자산\POOL별 관리자산\무차입자산\중복체크.xlsx')
중복

,계좌키,채무자키,POOL_
0,200919485,20407790,POOL_05
1,200919485,20407790,POOL_05
2,200919497,20407882,POOL_05
3,200919497,20407882,POOL_08
4,200919997,20407883,POOL_01
...,...,...,...
1725,201046260,20515407,POOL_13
1726,201046280,20515427,POOL_13
1727,201046280,20515427,POOL_13
1728,201046336,20515483,POOL_13


In [3]:
mate = True # 훌수
계좌키 = []
채무자키 = []
poola = []
poolb = []
for i, v in 중복.iterrows() :
    if v.계좌키 == "200922065" :
        print(i)
        print(i%2)
        print(v.POOL_)

    if i % 2 == 0 : 
        계좌키.append(v.계좌키)
        채무자키.append(v.채무자키)
        poola.append(v.POOL_)
        
    else :
        poolb.append(v.POOL_)
    
    
result = pd.DataFrame({"계좌키":계좌키, "채무자키":채무자키, "Pool_A":poola, "Pool_B":poolb})
result.to_excel(join(r"D:\3.자산\POOL별 관리자산\무차입자산","중복체크_1.xlsx"), index=False)